### 讀資料&調參

In [1]:
import csv
from datetime import datetime
from csv import DictReader
from math import exp, log, sqrt

# A, paths
data_path = '...'
train = data_path+'clicks_train.csv'               # path to training file
test = data_path+'clicks_test.csv'                 # path to testing file
submission = 'sub_proba.csv'  # path of to be outputted submission file

# B, model
alpha = .2  # learning rate
beta = .1   # smoothing parameter for adaptive learning rate
L1 = 0.1    # L1 regularization, larger value means more regularized
L2 = 0.1     # L2 regularization, larger value means more regularized

# C, feature/hash trick
D = 2 ** 20             # number of weights to use

# D, training/validation
epoch = 1       # learn training data for N passes
holdafter = None   # data after date N (exclusive) are used as validation
holdout = None  # use every N training instance for holdout validation
interaction = False     # whether to enable poly2 feature interactions

### promoted_content
* 把csv轉成prcont_dict

In [2]:
import pandas as pd 
pd.read_csv(data_path + "promoted_content.csv").head(3)

,ad_id,document_id,campaign_id,advertiser_id
0,1,6614,1,7
1,2,471467,2,7
2,3,7692,3,7


In [3]:
with open(data_path + "promoted_content.csv") as infile:
	prcont = csv.reader(infile)
	prcont_header = next(prcont)[1:]#['document_id', 'campaign_id', 'advertiser_id']

	prcont_dict = {}#把csv轉乘dict格式
	for ind,row in enumerate(prcont):
		prcont_dict[int(row[0])] = row[1:]
	print(len(prcont_dict))
del prcont

559583


### Event
* 把csv轉成event_dict

In [4]:
with open(data_path + "events.csv") as infile:
	events = csv.reader(infile)
	#events.next()
	next(events)
	event_header = ['uuid', 'document_id', 'platform', 'geo_location', 'loc_country', 'loc_state', 'loc_dma']
	event_dict = {}
	for ind,row in enumerate(events):
		tlist = row[1:3] + row[4:6]
		loc = row[5].split('>')
        
        #有些國家長度不同，要補齊
		if len(loc) == 3:
			tlist.extend(loc[:])
		elif len(loc) == 2:
			tlist.extend( loc[:]+[''])
		elif len(loc) == 1:
			tlist.extend( loc[:]+['',''])
		else:
			tlist.append(['','',''])	
		event_dict[int(row[0])] = tlist[:] 
	print(len(event_dict))
del events

23120126


### FTRL class

In [5]:
class ftrl_proximal(object):
    def __init__(self, alpha, beta, L1, L2, D, interaction):
        # parameters
        self.alpha = alpha; self.beta = beta 
        self.L1 = L1; self.L2 = L2

        # feature related parameters
        self.D = D; self.interaction = interaction
        # model
        self.n = [0.] * D        # n: squared sum of past gradients
        self.z = [0.] * D        # z: weights
        self.w = {}              # w: lazy weights
        
    def _indices(self, x):
        # first yield index of the bias term
        yield 0
        # then yield the normal indices
        for index in x:
            yield index

        # now yield interactions (if applicable)
        if self.interaction:
            D = self.D
            L = len(x)

            x = sorted(x)
            for i in xrange(L):
                for j in xrange(i+1, L):
                    # one-hot encode interactions with hash trick
                    yield abs(hash(str(x[i]) + '_' + str(x[j]))) % D

    def predict(self, x):
        ''' Get probability estimation on x
            INPUT: x: features
            OUTPUT: probability of p(y = 1 | x; w)
        '''
        # parameters
        alpha = self.alpha;beta = self.beta
        L1 = self.L1;L2 = self.L2
        # model
        n = self.n;z = self.z;w = {}
        # wTx is the inner product of w and x
        
        wTx = 0.
        for i in self._indices(x): #先return一個0,再把x的index一個一個放到i
            sign = -1. if z[i] < 0 else 1.  #z[x:varable index]

            # build w on the fly using z and n, hence the name - lazy weights
            # we are doing this at prediction instead of update time is because
            # this allows us for not storing the complete w
            
            #權重過小，把w[i]設為0
            if sign * z[i] <= L1:
                # w[i] vanishes due to L1 regularization
                w[i] = 0.
            else:
                # apply prediction time L1, L2 regularization to z and get w
                w[i] = (sign * L1 - z[i]) / ((beta + sqrt(n[i])) / alpha + L2)

            wTx += w[i]#得到wTx的加總值

        # cache the current w for update stage
        self.w = w
        
        # Return一個有上界的sigmoid function
        # bounded sigmoid function, this is the probability estimation
        return 1. / (1. + exp(-max(min(wTx, 35.), -35.)))

    def update(self, x, p, y):
        ''' Update model using x, p, y

            INPUT:
                x: feature, a list of indices
                p: click probability prediction of our model
                y: answer

            MODIFIES:
                self.n: increase by squared gradient
                self.z: weights
        '''

        # parameter
        alpha = self.alpha

        # model
        n = self.n
        z = self.z
        w = self.w

        # gradient under logloss
        g = p - y 

        # update z and n
        for i in self._indices(x):
            sigma = (sqrt(n[i] + g * g) - sqrt(n[i])) / alpha
            z[i] += g - sigma * w[i]
            n[i] += g * g

### Define Logloss function

In [6]:
#Done!
def logloss(p, y):
    ''' FUNCTION: Bounded logloss
        INPUT:
            p: our prediction
            y: real answer
        OUTPUT:
            logarithmic loss of p given y
    '''
    p = max(min(p, 1. - 10e-15), 10e-15)
    print(p)
    if y == 1.: #p(y=1∣x)
        return -log(p)  
    else: #p(y=0∣x)
        return -log(1. - p)

### Define Data Generator function

In [7]:
#Done: Data_generator
def data(path, D):
    ''' GENERATOR: Apply hash-trick to the original csv row
                   and for simplicity, we one-hot-encode everything
        INPUT:
            path: path to training or testing file
            D: the max index that we can hash to
        YIELDS:
            ID: id of the instance, mainly useless
            x: a list of hashed and one-hot-encoded 'indices'
               we only need the index since all values are either 0 or 1
            y: y = 1 if we have a click, else we have y = 0
    '''
    for t, row in enumerate(DictReader(open(path))):
        # 當前display_id,ad_id
        disp_id = int(row['display_id'])
        ad_id = int(row['ad_id'])

        # Y值
        y = 0.
        if 'clicked' in row:
            if row['clicked'] == '1':
                y = 1.
            del row['clicked']
        
        #紀錄x的hash位置
        #整除取餘數的概念是想hash多少維度
        x = []
        for key in row:
            x.append(abs(hash(key + '_' + row[key])) % D)

        #ad_id在各promoted_content(哪個商家、哪個檔期)下的變數
        #prcont_dict[ad_id]:[document_id	campaign_id	advertiser_id]
        row = prcont_dict.get(ad_id, [])		
        # build x
        ad_doc_id = -1
        for ind, val in enumerate(row):
            if ind==0:
                ad_doc_id = int(val)
            x.append(abs(hash(prcont_header[ind] + '_' + val)) % D)#abs(hash(string(Variable_value))) % D

        #display_id在各event(使用者行為：登入平台、觀看文章、地點)下的變數
        #event_header[display_id]:	uuid	document_id	timestamp	platform	geo_location
        row = event_dict.get(disp_id, [])
        ## build x
        disp_doc_id = -1
        for ind, val in enumerate(row):
            if ind==0:
                uuid_val = val
            if ind==1:
                disp_doc_id = int(val)
            x.append(abs(hash(event_header[ind] + '_' + val)) % D)#abs(hash(string(Variable_value))) % D

        ## yield 第幾筆資料,disp_id, ad_id, 
        #hashvalue(x):list[display_id,ad_id,document_id,campaign_id,advertiser_id,
        #                  uuid,	document_id	timestamp	platform	geo_location], y
        yield t, disp_id, ad_id, x, y

## Let's start training!

In [8]:
start = datetime.now()
# initialize ourselves a learner
learner = ftrl_proximal(alpha, beta, L1, L2, D, interaction)

In [9]:
for e in range(epoch):
    loss = 0.
    count = 0
    date = 0

    for t, disp_id, ad_id, x, y in data(train, D):  # data is a generator
        #    t: just a instance counter
        # date: you know what this is
        #   ID: id provided in original data
        #    x: features
        #    y: label (click)

        # step 1, get prediction from learner
        p = learner.predict(x)

        if (holdafter and date > holdafter) or (holdout and t % holdout == 0):
            # step 2-1, calculate validation loss
            #           we do not train with the validation data so that our
            #           validation loss is an accurate estimation
            #
            # holdafter: train instances from day 1 to day N
            #            validate with instances from day N + 1 and after
            #
            # holdout: validate with every N instance, train with others
            loss += logloss(p, y)
            count += 1
        else:
            # step 2-2, update learner with label (click) information
            learner.update(x, p, y)

## Predict test data

In [26]:
with open(submission, 'w') as outfile:
    outfile.write('display_id,ad_id,clicked\n')
    for t, disp_id, ad_id, x, y in data(test, D):
        p = learner.predict(x)
        outfile.write('%s,%s,%s\n' % (disp_id, ad_id, str(p)))

## Transform to submit format

In [27]:
import pandas as pd 
t = pd.read_csv(data_path+'sub_proba.csv')

得到test data FTRL預測機率值

In [28]:
t.head(3)

,display_id,ad_id,clicked
0,16874594,66758,0.052451
1,16874594,150083,0.086925
2,16874594,162754,0.203556


In [29]:
t.sort_values(['display_id','clicked'], inplace=True, ascending=False)
subi = t.groupby('display_id').ad_id.apply(lambda x: " ".join(map(str,x))).reset_index()

In [30]:
del t

In [31]:
len(subi)

6245533

In [32]:
subi.to_csv(data_path+'sub_format_FTRL.csv',index=False)